<a href="https://colab.research.google.com/github/HAVIGILI/OOD-Co-Act/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip uninstall -y accelerate

%pip install numpy==1.24.3
%pip install --quiet gdown

# Install OpenMIM tool
%pip install -U openmim

# Install a compatible PyTorch (for example, 2.0.0+cu118)
%pip install torch==2.0.0+cu118 torchvision==0.15.0+cu118 -f https://download.pytorch.org/whl/cu118/torch_stable.html

%pip install torchmetrics

# Then install mmcv-full pre-built for that combination:
!mim install "mmcv-full==1.7.2" -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html

Looking in links: https://download.pytorch.org/whl/cu118/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 926.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [2]:
!git clone -b 0.x --depth 1 https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .


Cloning into 'mmsegmentation'...
remote: Enumerating objects: 1405, done.
remote: Counting objects: 100% (1405/1405), done.
remote: Compressing objects: 100% (829/829), done.
remote: Total 1405 (delta 726), reused 892 (delta 556), pack-reused 0 (from 0)
Receiving objects: 100% (1405/1405), 9.06 MiB | 32.54 MiB/s, done.
Resolving deltas: 100% (726/726), done.
/content/mmsegmentation
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.8/648.8 kB 12.0 MB/s eta 0:00:00
  Running setup.py develop for mmsegmentation


In [3]:
import os
import torch
from mmseg.apis import init_segmentor, inference_segmentor
import mmcv
from IPython.display import Image, display
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
# Fishyscapes importet below. Keep this commented out

# RoadAnomaly21:
# !unzip -q /content/dataset_RoadAnomalyTrack.zip -d RoadAnomaly
# !unzip -q /content/gtFine_trainvaltest.zip -d gtFine_trainvaltest
# !wget -q http://wwwlehre.dhbw-stuttgart.de/~sgehrig/lostAndFoundDataset/leftImg8bit.zip
# !unzip -q leftImg8bit.zip -d /content/dataset_root/
# !wget -q http://robotics.ethz.ch/~asl-datasets/Fishyscapes/fishyscapes_lostandfound.zip
# !unzip -q fishyscapes_lostandfound.zip -d /content/dataset_root/

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# To get fishyscapes ground truth and pictures.

LEFTZIP_ID = "1BihdEUYOCRKpLfo8VvajTbuUtV3RyFtH"
FISHYZIP_ID = "1aR_qSjuykKWuKvM1dayecfb37cNYfZC3"

# download both
!gdown https://drive.google.com/uc?id=$LEFTZIP_ID -O leftImg8bit.zip
!gdown https://drive.google.com/uc?id=$FISHYZIP_ID -O fishyscapes.zip

# unzip into the exact paths your code expects
!unzip -q leftImg8bit.zip -d /content/drive/MyDrive/
!unzip -q fishyscapes.zip -d /content/drive/MyDrive/

images_root      = "/content/drive/MyDrive/leftImg8bit"
annotations_root = "/content/drive/MyDrive/fishyscapes_lostandfound"

Downloading...
From (original): https://drive.google.com/uc?id=1BihdEUYOCRKpLfo8VvajTbuUtV3RyFtH
From (redirected): https://drive.google.com/uc?id=1BihdEUYOCRKpLfo8VvajTbuUtV3RyFtH&confirm=t&uuid=c6d34d48-2cc7-4979-980e-2361f7406e7e
To: /content/mmsegmentation/leftImg8bit.zip
100% 5.80G/5.80G [00:31<00:00, 185MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aR_qSjuykKWuKvM1dayecfb37cNYfZC3
To: /content/mmsegmentation/fishyscapes.zip
100% 735k/735k [00:00<00:00, 158MB/s]
replace /content/drive/MyDrive/leftImg8bit/train/03_Hanns_Klemm_Str_19/03_Hanns_Klemm_Str_19_000003_000100_leftImg8bit.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace /content/drive/MyDrive/fishyscapes_lostandfound/0001_04_Maurener_Weg_8_000000_000090_labels.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
# Matching images with ground truth for fishyscapes.

import os

matched_annotations = []
matched_images = []
annotated_images = [f[5:].replace("_labels.png", "_leftImg8bit.png") for f in os.listdir(annotations_root)]
ood_annotations_paths = [os.path.join(annotations_root, f) for f in os.listdir(annotations_root)]

# Walk through the image directory
for dir_path, dir_names, filenames in os.walk(images_root):
        for filename in filenames:
            if filename in annotated_images:
                index = annotated_images.index(filename)
                annotation_path = ood_annotations_paths[index]
                image_path = os.path.join(dir_path, filename)
                matched_annotations.append(annotation_path)
                matched_images.append(image_path)
print("Matched pairs:")
for annotation, image in zip(matched_annotations, matched_images):
    print("annotation", annotation, "matched with", image)


Matched pairs:
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0095_13_Elly_Beinhorn_Str_000002_000120_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000002_000120_leftImg8bit.png
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0088_13_Elly_Beinhorn_Str_000000_000290_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000000_000290_leftImg8bit.png
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0092_13_Elly_Beinhorn_Str_000001_000230_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000001_000230_leftImg8bit.png
annotation /content/drive/MyDrive/fishyscapes_lostandfound/0089_13_Elly_Beinhorn_Str_000001_000080_labels.png matched with /content/drive/MyDrive/leftImg8bit/train/13_Elly_Beinhorn_Str/13_Elly_Beinhorn_Str_000001_000080_leftImg8bit.png
annotation /content/drive/MyDrive/fishysc

In [8]:
# Initialize the segmentation model (DeepLabV3+ pretrained on Cityscapes)
config_file = 'configs/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes.py'
checkpoint_file = ('https://download.openmmlab.com/mmsegmentation/v0.5/'
                   'deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/'
                   'deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth')
model = init_segmentor(config_file, checkpoint_file, device='cuda:0' if torch.cuda.is_available() else 'cpu')

/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from http path: https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth


Downloading: "https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth
100%|██████████| 239M/239M [00:07<00:00, 35.3MB/s]


In [9]:
# ──────────────────────────────────────────────
import os
from typing import List, Tuple

import torch
from mmseg.apis import init_segmentor          # ← plain MMSeg import
from calibrator import Calibrator
from anomalydetector import AnomalyDetector

# --------------------------------------------------------------------
# 1.  Collect training image / GT paths  (unchanged helper)
# --------------------------------------------------------------------
def cityscapes_paths(img_root: str, gt_root: str) -> Tuple[List[str], List[str]]:
    images, gts = [], []
    for city in sorted(os.listdir(img_root)):
        d_img, d_gt = os.path.join(img_root, city), os.path.join(gt_root, city)
        if not os.path.isdir(d_img):
            continue
        for fn in sorted(os.listdir(d_img)):
            if not fn.endswith('_leftImg8bit.png'):
                continue
            images.append(os.path.join(d_img, fn))
            label_fn = fn.replace('_leftImg8bit.png', '_gtFine_labelIds.png')
            gts.append(os.path.join(d_gt, label_fn))
    return images, gts


img_root = '/content/drive/MyDrive/leftImg8bit_trainvaltest/leftImg8bit/train'
gt_root  = '/content/drive/MyDrive/gtFine_trainvaltest/gtFine/train'
images, gts = cityscapes_paths(img_root, gt_root)
print(f'Found {len(images)} images')

# --------------------------------------------------------------------
# 2.  Bring up the MMSeg model  (one line)
# --------------------------------------------------------------------
CFG  = 'configs/deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes.py'
CKPT = ('https://download.openmmlab.com/mmsegmentation/v0.5/'
        'deeplabv3plus/deeplabv3plus_r101-d8_512x1024_40k_cityscapes/'
        'deeplabv3plus_r101-d8_512x1024_40k_cityscapes_20200605_094614-3769eecf.pth')

DEV = 'cuda:0' if torch.cuda.is_available() else 'cpu'
mmseg_model = init_segmentor(CFG, CKPT, device=DEV)   # <-- raw MMSeg model

# --------------------------------------------------------------------
# 3.  Calibrate (or reload previous statistics)
# --------------------------------------------------------------------
SAVE = '/content/drive/MyDrive/calibrator_state_ground_truth.pt'

if os.path.exists(SAVE):
    # one-liner restore: builds a *fresh* model internally
    calib = Calibrator.load(SAVE, mmseg_cfg=CFG, mmseg_ckpt=CKPT, device=DEV)
else:
    calib = Calibrator(mmseg_model)    # hand the model in
    calib.run(images, gts)             # long pass over the dataset
    calib.save(SAVE)


In [ ]:
# ╔════════════════════════════════════════╗
# ║  Grid-search for LINe + Co-activation  ║
# ╚════════════════════════════════════════╝

import itertools
import gc

start, end = 0, 100     # Max 100
matched_images_short      = matched_images[start:end]
matched_annotations_short = matched_annotations[start:end]
images_to_be_plotted      = []  # show figures only for the first image if 0, can use 3:7 to plot image 4-8 for example. Empty for no figures

# Hyper-parameter lists (edit to sweep more values)
activation_clippings            = [100]
activation_prunings             = [0]
weight_prunings                 = [0]
temperatures_line               = [1]

inverse_convert_to_ones         = [False, True]
binary_or_not                   = [False, True]
wt_thresholds                   = [0, 0]
wt_u_thresholds                 = [1, 0.1]
temperatures_co                 = [0.3, 1]

baseline_and_line_blur_ksizes   = [(3, 3)]
baseline_and_line_sigmas        = [0]

co_blur_sizes                   = [23]
co_sigmas                       = [5]
max_pools                       = [11]

clips                           = [(0, 1000000), (250000, 10000000), (0, 0)]
co_weighteds                    = [0]
weight_modes                    = ['softmax']
use_c_u_ratios                  = [False, True]

plot_pixel_acts   = True
plot_pair_ratios  = True
id_dot, ood_dot   = (850, 1900, 'lime'), (485, 1360, 'red') # Place these manually by choosing coordinates on a ood object and an id object.

state = calib.state_dict()


grid = {
    "temp_co":                temperatures_co,
    "activation_clipping":    activation_clippings,
    "activation_pruning":     activation_prunings,
    "weight_pruning":         weight_prunings,
    "wt_pair":                list(zip(wt_thresholds, wt_u_thresholds)),
    "co_blur_pair":           list(zip(co_blur_sizes, co_sigmas)),
    "clip_val":               clips,
    "temp_line":              temperatures_line,
    "co_weighted":            co_weighteds,
    "inv2ones":               inverse_convert_to_ones,
    "bin_or_not":             binary_or_not,
    "baseline_pair":          list(zip(baseline_and_line_blur_ksizes, baseline_and_line_sigmas)),
    "max_pool":               max_pools,
    "weight_mode":            weight_modes,
    "c_u_ratio":              use_c_u_ratios,
}

param_names, param_values = zip(*grid.items())

# 4. Tracking best metrics
best_ap    =  -1
best_fpr   =   1
best_auc   =  -1
best_tag_ap  = best_tag_fpr = best_tag_auc = None

# 5. Iterate over the Cartesian product of all hyper-params
for combo in itertools.product(*param_values):
    P = dict(zip(param_names, combo))

    # unpack zipped pairs
    wt_thr,  wt_thr_u      = P["wt_pair"]
    co_blur_ksize, co_sigma = P["co_blur_pair"]
    baseline_blur, baseline_sigma = P["baseline_pair"]

    # create & configure detector
    det = AnomalyDetector(matched_images_short,
                          matched_annotations_short,
                          model,
                          state)
    det.plot_pixel_activations = plot_pixel_acts
    det.plot_pair_ratios       = plot_pair_ratios
    det.images_to_be_plotted   = images_to_be_plotted
    det.id_dot, det.ood_dot    = id_dot, ood_dot

    # set thresholds & params
    det.activation_clipping     = P["activation_clipping"]
    det.activation_pruning      = P["activation_pruning"]
    det.weight_pruning          = P["weight_pruning"]
    det.temperature_co          = P["temp_co"]
    det.temperature_line        = P["temp_line"]
    det.inverse_convert_to_ones = P["inv2ones"]
    det.make_feat_binary        = P["bin_or_not"]
    det.wt_threshold            = wt_thr
    det.wt_u_threshold          = wt_thr_u
    det.co_blur_ksize           = co_blur_ksize
    det.co_sigma                = co_sigma
    det.blur_ksize              = baseline_blur
    det.sigma                   = baseline_sigma
    det.max_pool                = P["max_pool"]
    det.clips                   = P["clip_val"]
    det.co_weighted             = P["co_weighted"]
    det.weight_mode             = P["weight_mode"]
    det.c_u_ratio               = P["c_u_ratio"]

    # run inference
    det.ood_inference()
    IDs, ood_scores = det.get_ood_score_lists()

    # evaluate selected methods
    methods = {
        "ood_scores_coact_bin":       "Co-act count",
        "ood_scores_coact_wt":        "Co-act magnitude",
        "ood_scores_coact_any":       "Co-act any",
        "ood_scores_nr_activation":   "#activations",
    }

    # build a reusable run tag
    base_tag = " | ".join(f"{k}={P[k]}" for k in param_names)

    scores_line = ood_scores["ood_scores_line"]
    for method, title in methods.items():
        scores = -ood_scores[method] / scores_line / scores_line / scores_line
        scores = (scores - scores.min()) / (scores.ptp() + 1e-9)

        run_tag = f"method | ={method}{base_tag}"
        print(run_tag)

        auc, fpr, ap = det.calculate_metrics(IDs, scores,
                                             plot_hist=False,
                                             title=title)

        if ap  > best_ap:  best_ap,  best_tag_ap  = ap,  run_tag
        if fpr < best_fpr: best_fpr, best_tag_fpr = fpr, run_tag
        if auc  > best_auc: best_auc, best_tag_auc = auc, run_tag

    # cleanup GPU memory
    del det
    gc.collect()
    torch.cuda.empty_cache()

# 6. Print summary of best combinations
print("\n══════ BEST RESULTS ══════")
print(f"• Best AP    = {best_ap:.4f}  → {best_tag_ap}")
print(f"• Best FPR95 = {best_fpr:.4f}  → {best_tag_fpr}")
print(f"• Best AUROC = {best_auc:.4f}  → {best_tag_auc}")


method | =ood_scores_coact_bintemp_co=0.3 | activation_clipping=100 | activation_pruning=0 | weight_pruning=0 | wt_pair=(0, 1) | co_blur_pair=(23, 5) | clip_val=(0, 1000000) | temp_line=1 | co_weighted=0 | inv2ones=False | bin_or_not=False | baseline_pair=((3, 3), 0) | max_pool=11 | weight_mode=softmax | c_u_ratio=False
AUROC=0.9501  FPR95=0.3238  AP=0.2127
method | =ood_scores_coact_wttemp_co=0.3 | activation_clipping=100 | activation_pruning=0 | weight_pruning=0 | wt_pair=(0, 1) | co_blur_pair=(23, 5) | clip_val=(0, 1000000) | temp_line=1 | co_weighted=0 | inv2ones=False | bin_or_not=False | baseline_pair=((3, 3), 0) | max_pool=11 | weight_mode=softmax | c_u_ratio=False
AUROC=0.9575  FPR95=0.1893  AP=0.2036
method | =ood_scores_coact_anytemp_co=0.3 | activation_clipping=100 | activation_pruning=0 | weight_pruning=0 | wt_pair=(0, 1) | co_blur_pair=(23, 5) | clip_val=(0, 1000000) | temp_line=1 | co_weighted=0 | inv2ones=False | bin_or_not=False | baseline_pair=((3, 3), 0) | max_pool=11

In [15]:
# Set true and run if you need to free memory

delete = True
if delete:
    del det
    import gc
    gc.collect()
    import torch
    torch.cuda.empty_cache()